# Proxecto Exemplo: Predición de Prezos de Vivendas en California

**Obxectivo:** Construír un modelo de regresión lineal que prediga o prezo dunha vivenda en California baseándose nas características da zona onde se atopa.

Neste exercicio imos predicir o prezo medio dunha vivenda en función de características como os ingresos da zona, a antigüidade media das casas, o número medio de cuartos e a localización xeográfica. Usamos un modelo sinxelo de regresión lineal para comezar a explorar como funciona o Machine Learning cando tratamos datos reais

### Paso 1: Cargar os Datos (Directamente dende Scikit-learn)

Non necesitamos buscar ficheiros CSV. A librería `scikit-learn` ofrécenos o dataset directamente, o que facilita moito o comezo.

In [ ]:
# Importamos as librarías necesarias
import pandas as pd
from sklearn.datasets import fetch_california_housing
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from ipywidgets import FloatSlider, FloatText, Button, Label, VBox, interactive_output
from IPython.display import display
import numpy as np  

print("Libraries importes successfully!")

# Cargamos o conxunto de datos de vivendas de California
housing = fetch_california_housing()

# Convertemos os datos a un DataFrame de Pandas para traballar máis comodamente
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['PRICE'] = housing.target # O prezo está en unidades de 100,000$

# Botamos unha primeira ollada aos datos
print("Primeiras 5 filas do dataset:")
print(df.head())

### Paso 2: Análise e Visualización Exploratoria (Entender a Relación)

Antes de adestrar, é fundamental *ver* os datos. Imos visualizar a relación entre a característica que, a priori, parece máis influente, os **ingresos medios da veciñanza** (`MedInc`), e o **prezo** da vivenda.

In [ ]:
# Creamos un gráfico de dispersión para ver a relación
plt.figure(figsize=(6, 5))
sns.scatterplot(x='MedInc', y='PRICE', data=df, alpha=0.5)

plt.title('Prezo da Vivenda vs. Ingresos Medios da Zona', fontsize=16)
plt.xlabel('Ingresos Medios (en decenas de miles de $)', fontsize=12)
plt.ylabel('Prezo Mediano da Vivenda (en centos de miles de $)', fontsize=12)
plt.grid(True)
plt.show()

### Paso 3: Preparar os Datos e Adestrar o Modelo (Versión Simple)

Comezaremos coa versión máis sinxela posible: predicir o prezo usando **unha soa característica**, os ingresos medios (`MedInc`). Isto permitiranos debuxar unha liña de regresión en 2D moi fácil de interpretar.

In [ ]:
# 1. Definimos a nosa característica (X) e o noso obxectivo (y)
X = df[['MedInc']] 
y = df['PRICE']

# 2. Dividimos os datos para adestrar e para probar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Creamos e adestramos o modelo de regresión lineal
simple_model = LinearRegression()
simple_model.fit(X_train, y_train)

print("Modelo de regresión lineal simple adestrado con éxito!")

### Paso 4: Visualizar a Liña de Regresión (A Parte Máis Intuitiva)

Agora debuxaremos a "liña de mellor axuste" que o noso modelo aprendeu sobre os datos de proba. Esta liña representa a predición do modelo para calquera valor de ingresos medios.

In [ ]:
# Facemos predicións sobre os datos de proba
prediccions_simples = simple_model.predict(X_test)

# Debuxamos os datos de proba reais (puntos azuis)
plt.figure(figsize=(10, 8))
plt.scatter(X_test, y_test, alpha=0.5, label='Datos Reais')

# Debuxamos a nosa liña de regresión (liña vermella)
plt.plot(X_test, prediccions_simples, color='red', linewidth=3, label='Liña de Regresión do Modelo')

plt.title('Liña de Regresión: Axuste do Modelo aos Datos', fontsize=16)
plt.xlabel('Ingresos Medios', fontsize=12)
plt.ylabel('Prezo da Vivenda', fontsize=12)
plt.legend()
plt.grid(True)
plt.show()

print("A liña vermella é a predición do noso modelo. É a liña recta que mellor se axusta á nube de puntos, minimizando o erro colectivo.")

### Paso 5: Probamos o modelo

Introducimos un valor para a única característica que coñece o modelo, os "Ingresos Medios", e ao premer o botón, utiliza a liña de regresión que aprendeu para calcular o prezo estimado da vivenda.

In [ ]:
# Creamos os elementos do formulario
income_input = widgets.FloatText(description="Ingresos Medios:", value=3.5)
predict_button = widgets.Button(description="Predicir Prezo")
output_label = widgets.Label()

# Función para predecir
def on_predict_button_clicked(b):
    income_value = income_input.value
    predicted_price = simple_model.predict(np.array([[income_value]]))[0]
    final_price = predicted_price * 100000
    output_label.value = f"Prezo Predicido: ${final_price:,.2f}"

predict_button.on_click(on_predict_button_clicked)

# Amosamos o resultado
display(income_input, predict_button, output_label)

### Paso 6: (AVANZADO) Mapa de prezos das vivendas en California

Este código xera un mapa visual dos prezos da vivenda en California, debuxando cada casa como un punto segundo a súa lonxitude e latitude. A cor de cada punto está determinada polo prezo, o que permite identificar de forma rápida e intuitiva as zonas xeograficamente máis caras (con cores máis claras) fronte ás máis económicas (con cores máis escuras).

In [ ]:
# Amosamos un gráfico cos prezos baseándonos na situación da vivenda
plt.figure(figsize=(8, 6))
plt.scatter(df['Longitude'], df['Latitude'], c=df['PRICE'], cmap='viridis', alpha=0.5)
plt.colorbar(label='Prezo Mediano da Vivenda ($100,000s)')
plt.xlabel('Lonxitude')
plt.ylabel('Latitude')
plt.title('Mapa de Prezos das Vivendas en California')
plt.show()
print("No mapa pódense ver claramente os prezos altos preto da costa (ex: Os Ánxeles e a Baía de San Francisco).")

### Paso 6: Expandir a un Modelo de Regresión Múltiple

Para mellorar a precisión, podemos usar máis características. O proceso é case idéntico, pero agora o modelo terá en conta máis información para facer a súa predición. Non poderemos debuxar unha única liña (porque teriamos máis de dúas dimensións), pero podemos comparar o seu rendemento numericamente.

In [ ]:
# Escollemos as características que desexemos
features = ['Latitude', 'Longitude']
X = df[features]
y = df['PRICE']

# Dividimos os datos e adestramos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
final_model = LinearRegression()
final_model.fit(X_train, y_train)

print("Modelo final adestrado con éxito!")

### Paso 7: Probamos o modelo

Poñemos o modelo a proba: imos introducir manualmente as coordenadas (latitude e lonxitude) dun lugar específico de California para pedirlle ao noso modelo que estime o prezo dunha vivenda nesa localización.

In [ ]:
# Definimos os inpunts para variar a latitude e a lonxitude, ademais dunha etiqueta onde se amosará o prezo predito
lat_input = FloatText(value=34.0,description='Latitude:',disabled=False)
lon_input = FloatText(value=-118.0,description='Lonxitude:',disabled=False)

# Creamos o botón que activará o cálculo
calcular_button = Button(description='Calcular Prezo',button_style='success', tooltip='Fai clic para estimar o prezo da vivenda',icon='calculator')

# A etiqueta de saída segue a ser a mesma
output_label = Label(value="Introduce os valores e preme en 'Calcular'.")

# Creamos unha función que se executará SÓ cando se prema o botón
def on_button_clicked(b):
    # Dentro da función, collemos os valores actuais das caixas de texto
    lat = lat_input.value
    lon = lon_input.value
    
    # Executamos a mesma lóxica de predición que tiñas antes
    datos_para_predecir = [[lat, lon]]
    prezo_predicido = final_model.predict(datos_para_predecir)
    prezo_final = prezo_predicido[0] * 100000
    
    # Actualizamos o texto da etiqueta co resultado
    output_label.value = f'O prezo estimado da vivenda é: ${prezo_final:,.2f}'

# 4. Conectamos a función ao evento 'on_click' do botón
calcular_button.on_click(on_button_clicked)

# Poñemos todo xunto para que apareza ordenado
ui = VBox([lat_input, lon_input, calcular_button, output_label])
display(ui)